# Финальный проект. Неделя 1

 ###  Анализ тональности отзывов на фильмы: строим простые модели

In [34]:
from nltk.corpus import movie_reviews
import pandas as pd
import numpy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_score

Получение ИДшников позитивных и негативных отзывов

In [2]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

Список негативных отзывов

In [7]:
negfeats = [(movie_reviews.words(fileids=[f]),0) for f in negids]
posfeats = [(movie_reviews.words(fileids=[f]),1) for f in posids]

In [20]:
print negfeats[0]

([u'plot', u':', u'two', u'teen', u'couples', u'go', ...], 0)


Список со всеми отзывами

In [16]:
allfeats =negfeats+posfeats

In [17]:
len(allfeats)

2000

In [18]:
' '.join(map(str, (len(negfeats),len(posfeats))))

'1000 1000'

Как видим все отзывы размечены

    3. Создадим признаковое описание отзывов

In [26]:
model = CountVectorizer()
features = model.fit_transform(map(lambda x:' '.join(x[0]), allfeats))


In [30]:
#количество признаков
features.shape[1]

39659

    4.Создадим бейзлайн с Логистической регрессией с параметрами по умолчанию, и измерим точность и ROC-AUC на кросс-валидации с дефолтовыми параметрами

In [37]:
pipeline = Pipeline(steps=[('param', CountVectorizer()), ('logreg', LogisticRegression() )])
review = map(lambda x:' '.join(x[0]), allfeats)
labels = map(lambda x:x[1], allfeats)
score = cross_val_score(pipeline, review, labels, scoring='accuracy')
print score.mean()

0.836021650393


In [39]:
roc_score = cross_val_score(pipeline, review, labels, scoring='roc_auc')
print roc_score.mean()

0.910776493783


In [45]:
model_full = LogisticRegression()
model_full.fit(features, labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [52]:
feature_array = [(idx,coef) for idx,coef  in enumerate(model_full.coef_[0])]

In [57]:
sort_array = pd.DataFrame(data = feature_array, columns = ('item_id','item'))
sort_array.sort_values(['item'], ascending=False, inplace=True)
sort_array['item_id'].head().tolist()

[14159L, 15246L, 2908L, 27955L, 38610L]

In [ ]:
features.